# State-specific Choice Sets

This notebook discusses how we want to implement state spaces which are aware of dense dimension and flexible choice sets.

## Design

A `StateSpace` consists of multiple `SubStateSpace`s for each combination of dense dimension vector, period, and choice set. First, we have separated state spaces by dense dimension to avoid massive duplication. Secondly, we further want to divide state spaces by periods to make it more explicit. Implicitly, we already work with periodic state spaces. We also guess that many flexible choice sets are connected to periods (tracking system, retirement) and with periodic state spaces we can exploit the sparseness. Thirdly, we divide by choice sets such that `wages`, etc. contain only the most necessary information. This will speed up the Monte Carlo simulations and make the inadmissiblity penalty redundant for the full solution.

For example, the model `kw_94_one` has one `SubStateSpace` for each period from 0 to 9. Starting with period 10 some states have reached maximum years of schooling and have only three available alternatives. Thus, for each period from 10 and on, there are two `SubStateSpace`s for each period. All `SubStateSpace`s are captured in one `StateSpace` class which manages the access and communication (continuation values) between the `SubStateSpaces`.

The `StateSpace` captures all subordinate elements. It comprises ..

- of functions to get periodical information, continuation values, etc..
- A list of parts of the core state space divided by periods and choice sets (not defined by dense vectors).
- A dictionary of dense vectors which contain their dense covariates.

Every `SubStateSpace` has its ...

- index of a period-choice-set-specific part of the core state space. We can only have an index, otherwise we would duplicate the core state space across all dense dimension which we wanted to avoid in the first place.
- index for a specific dense vector plus its dense covariates. (Does not have to be an index, we can also have a copy)
- available choices to efficiently generate the rewards.

Let us start with the implementation of the state space.

In [15]:
import numpy as np
import pandas as pd
import respy as rp
import itertools
import copy

from respy.pre_processing.model_processing import process_params_and_options
from respy.state_space import _create_core_and_indexer, create_state_space_class
from respy.pre_processing.process_covariates import separate_covariates_into_core_dense_mixed

### Load the example model

No exogenous processes for now.

In [16]:
# Load model.
params, options = rp.get_example_model("robinson_crusoe_extended", with_data=False)

# Extend with observable characteristic.
params.loc[("observable_health_well", "probability"), "value"] = 0.9
params.loc[("observable_health_sick", "probability"), "value"] = 0.1

# Sick people can never work.
options["inadmissible_choices"] = {
    "fishing": ["health == 'sick'"],
}

# Create internal specification objects.
optim_paras, options_ = process_params_and_options(params, options)


In [17]:
options_ = separate_covariates_into_core_dense_mixed(options_, optim_paras)

### Pre Processing

We want to split the inadmissability contraints in three parts. 
1. options["inadmissable_choices_core"]: 
These are choice restrictions that just depend on the core state! 
- We will apply these choices to create choice sets for each core state. 

2. options["inadmissable_choices_dense"]:
- We will create all dense combinations and we will assign all of these combinations admissable choice sets

3. options["inadmissable_choices_mixed"]
- We will write a function that transforms the mixed constraints to a constraint on a part of the core sp with a fixed dense comb! 

In [18]:
def separate_choice_restrictions_into_core_dense_mixed(options, optim_paras):
    """
    This is the simplest version we can possbly think of!
    #TODO: Align with separation of covariates. If possible unify both.
    If not try to synchronize as far as possible!
    """
    options = copy.deepcopy(options)
    covariates = options["covariates"]
    
    # Add ne dict keys
    constr_list = list()
    for choice in options["inadmissible_choices"].keys():
        for choice_constr in options["inadmissible_choices"][choice]:
            if any(x in choice_constr for x in options["covariates_dense"]) is False:
                constr_list.append((choice_constr, choice, "core"))    
            elif any(x in choice_constr for x in options["covariates_core"]) is False:
                constr_list.append((choice_constr, choice, "dense"))
            else:
                constr_list.append((choice_constr, choice, "mixed"))        
    
    for sp in ["core", "dense", "mixed"]:
        options[f"inadmissible_choices_{sp}"] = {}
        for choice in options["inadmissible_choices"].keys():
            relevant_contraints = [ x for x in constr_list if x[1]==choice and x[2]==sp]
            if relevant_contraints == []:
                pass
            else:
                options[f"inadmissible_choices_{sp}"][choice] = relevant_contraints
    return options



In [ ]:
options_ = separate_choice_restrictions_into_core_dense_mixed(options, optim_paras)

### Create object for the initialization of the sub state spaces

We want to define methods that alow us to create objects that we can 
base our state space construction upon! 

Therefore we will create the core Sp as before. We will use the choice restrictions to create a dict that contains a choice/period comb as keys and the corresponding part of the core sp df as value!

We will also apply constraints to the dense sp!

Therafter we will combine the information and apply mixed constraints. 
In the end we want a dict that contains (period, choice_set, dense_point): np.array([indexes])




In [251]:
def _create_choice_sets(df, optim_paras, options, category):
    """
    Todo: Check to which extent we can use the same procedure for 
    core and dense
    """
    df = df.copy()
    
    # Apply user-defined constraints
    for choice in optim_paras["choices"]:
        df[choice] = False
    
    for choice in options[f"inadmissible_choices_{category}"].keys():
        for formula in options[f"inadmissible_choices_{category}"][choice]:
            try: 
                df[choice] |= df.eval(formula)
            except pd.core.computation.ops.UndefinedVariableError:
                pass
    return df

def _split_core_ssp(core):
    """
    This function splits the sp according to period and choice set
    """
    periodic_cores = {idx: sub for idx, sub in core_.groupby("period")}
    periodic_choice_cores = {(period, choice_set): sub 
                           for period in periodic_cores
                           for choice_set, sub 
                           in periodic_cores[period].groupby(list(optim_paras["choices"]))
                          }
    return periodic_choice_cores


def _get_subspace_of_choice(choice_set):
    out = []
    choice_set = np.array(choice_set)
    check = np.where(choice_set)
    for x in itertools.combinations_with_replacement([True, False],len(choice_set)):
        if all(y is True for y in np.array(x)[check]):
            out.append(x)
        else:
            pass
    return out

    
def _create_base_sp(period_choice_cores, dense_df, options, optim_paras):
    """
    This function should directly give us a number of indices after all!
    TODO: Major cleanup. Way to complicated for now! 
    """
    state_space = {}
    for (period, choice_set), df in period_choice_cores.items():
        for dense_state in dense_df.index:
            admissible_choices = _get_subspace_of_choice(
                dense_df.loc[dense_state,list(optim_paras["choices"].keys())].to_numpy())
            
            if choice_set in admissible_choices: 
                df_ = df.copy().assign(
                    **dense_df.drop(
                        columns=optim_paras["choices"].keys()).loc[dense_state].to_dict())
                df_ = _create_choice_sets(
                    df_,
                    optim_paras,
                    options,
                    "mixed"
                )
               
                for choice_set_, sub_df in df_.groupby(list(optim_paras["choices"])):
                    #Here we want to assign an index
                    state_space[(period, choice_set_, dense_state)] = sub_df.index
    return state_space


In [255]:
state_space = _create_base_sp(period_choice_cores, dense_, options_, optim_paras)

### Plug things together!

Duplicate each part with all dense vectors and re-compute `is_inadmissible` with the dense information. This is necessary because observables may block a choice. This gives us the final combination between dense vectors, choice sets and parts of the core state space.

In [12]:
core, indexer = _create_core_and_indexer(optim_paras, options_)

[(0,), (1,)]

In [14]:
def create_sp():
    """
    Define a function that calls all the prior methods to create something 
    that looks like a sp! 
    """
    # We create the full core first! (Do we have to create covariates here?)
    core, indexer = _create_core_and_indexer(optim_paras, options_)
    
    #We apply all inadmissable states 
    core = _create_choice_sets(core, options, optim_paras, "core")
    period_choice_cores = _split_core_ssp
    
    #Create dense sp!
    dense_grid = rp.state_space._create_dense_state_space_grid(optim_paras)
    dense_cov = rp.state_space._create_dense_state_space_covariates(dense_grid, optim_paras, options_)
    dense_df = pd.DataFrame(dense_cov).transpose()
    dense_choice_sets = _create_choice_sets(dense_df, options, optim_paras, "dense")
    
    #Plug both together
    
    
    #Initialize all series
    #Call classes below 
    

NameError: name 'period_choice_cores' is not defined

### The state space indices

This is the base object that contains most of the information to properly seperate the information between different class instances!

In [11]:
state_space

{(0, (True, False, False), (0,)): (0, (False, False, False)),
 (0, (False, False, False), (1,)): (0, (False, False, False)),
 (1, (True, False, False), (0,)): (1, (False, False, False)),
 (1, (False, False, False), (1,)): (1, (False, False, False)),
 (2, (True, False, False), (0,)): (2, (False, False, False)),
 (2, (False, False, False), (1,)): (2, (False, False, False)),
 (3, (True, False, False), (0,)): (3, (False, False, False)),
 (3, (False, False, False), (1,)): (3, (False, False, False)),
 (4, (True, False, False), (0,)): (4, (False, False, False)),
 (4, (False, False, False), (1,)): (4, (False, False, False)),
 (5, (True, False, False), (0,)): (5, (False, False, False)),
 (5, (False, False, False), (1,)): (5, (False, False, False)),
 (6, (True, False, False), (0,)): (6, (False, False, False)),
 (6, (False, False, False), (1,)): (6, (False, False, False)),
 (7, (True, False, False), (0,)): (7, (False, False, False)),
 (7, (False, False, False), (1,)): (7, (False, False, False)),


### The SubStateSpace

The idea is that the the state space has three levels: 
- _MultiDimStateSpace: This is the highest level. Takes the core, the indexer and the period_choice_dense core indexer. It then just creates one instance of the lower class for each dense covariate and distributes information accordingly. (Not a lot changes here)
-_SingleDimStateSpace: This state space manages one single core sp. 
It creates one instance of the lower class for each choice set that appears and distributes information accordingly. 
This object has a different role than before. Importantly some attributes are also changed by operations from the lower classes!
-ChoiceSetSpace:
This class is the lowest object. It receives the indices of core states that belong to each choice set in every period. 
Al information like wages is assigned to this lowest class since it is only processed there. For continuation values the class however draws on the SingleDimsSP since it is possible that child states have differet choice sets

In [ ]:
#We try to get the easiest workflow!
#This will inplace change the cont value array
class _ChoiceSetSpace(_BaseStateSpace):
    """
    This is just a bundle of methods that should calculate 
    """
    def __init__(
    choice_set,
    period_cores,
    continuation_values, 
    base_draws_sol,
    optim_paras,
    options,
    dense_dim=None,
    dense_covariates=None,
    is_inadmissible=None,
    indices_of_child_states=None,
    slices_by_periods=None,
    
        ):
        self.dense = dense 
        self.choice_set = choice_set
        self.num_choices = choice_set.count(True)
        self.core_dict = core_dict
        

    def get_continuation_values(self, period, indices=None):
        """
        For now only works with explicit period! 
        """
        n_periods = len(self.indexer)

        if period == n_periods - 1:
            last_slice = self.period_cores[-1]
            n_states_last_period = len(last_slice)
            continuation_values = np.zeros((n_states_last_period, self.n_choices))

        else:
            child_indices = self.get_attribute("indices_of_child_states")[self.period_cores[period]][:,choice_set]

            mask = child_indices != INDEXER_INVALID_INDEX
            valid_indices = np.where(mask, child_indices, 0)
            continuation_values = np.where(
                mask, self.get_attribute("expected_value_functions")[valid_indices], 0
            )
        continuation_values[:,self.choice_set]
        
        return continuation_values
    
    def get_attribute(self,attr):
        return getattr(self, attr)
    
    def set_attribute(self,attr,value):
        self.get_attribute(attribute)[:] = value

        

In [17]:
class _SingleDimsSpace(_BaseStateSpace):
    """
    We should initialize with child states!
    Cont values are something we need tp get the super class for!
    """

    def __init__(
        self,
        core,
        period_choice_cores,
        indexer,
        base_draws_sol,
        optim_paras,
        continuation_values,
        options,
        dense_dim=None,
        dense_covariates=None,
        is_inadmissible=None,
        indices_of_child_states=None,
        slices_by_periods=None,
    ):
        self.dense_dim = dense_dim
        self.core = core
        self.period_choice_cores = period_choice_cores 
        self.indexer = indexer
        self.dense_covariates = dense_covariates if dense_covariates is not None else {}
        self.mixed_covariates = options["covariates_mixed"]
        self.base_draws_sol = base_draws_sol
        self.slices_by_periods = (
            super()._create_slices_by_core_periods()
            if slices_by_periods is None
            else slices_by_periods
        )
        self._initialize_attributes(optim_paras)
        self.is_inadmissible = (
            self._create_is_inadmissible(optim_paras, options)
            if is_inadmissible is None
            else is_inadmissible
        )
        self.indices_of_child_states = (
            super()._create_indices_of_child_states(optim_paras)
            if indices_of_child_states is None
            else indices_of_child_states
        )
        self.choice_spaces = {choice_set: _ChoiceSetSpace(
        choice_set, [period_choice_cores[dense,period, choice_set]) for period in option["n_periods"]]) for choice_set in }
    
    def get_continuation_values(self,period):
        return {
            key: sss.get_continuation_values(period, indices)
            for key, sss in self.choice_spaces.items()
        }
        
    def get_attribute(self, attr):
        """Get an attribute of the state space."""
        return getattr(self, attr)

    
    def get_attribute_from_period_choice(self, attribute, period, choice):
        attr = self.get_attribute(attribute)
        slice_ = self.cores[(period, choice)]
        out = attr[slice_]
        
        return out
        
    def set_attribute(self, attribute, value):
        self.get_attribute(attribute)[:] = value
    
    def set_attribute_choice_sets_private(self, attribute, value):
        for key,ssp in self.choice_spaces.items():
            ssp.set_attribute(self, attribute, value[key])

    def set_attribute_from_period_choice_set(self, attribute, value, period):
        self.get_attribute_from_period_choice(attribute, period)[:] = value
    
    @property
    def states(self):
        states = self.core.copy().assign(**self.dense_covariates)
        states = compute_covariates(states, self.mixed_covariates)
        return states



In [ ]:
class _MultiDimStateSpace(_BaseStateSpace):
    """The state space of a discrete choice dynamic programming model.
    This class wraps the whole state space of the model.
    In this case we take the whole seperated choice space and create individual objects for each 
    dense option. Operations are still seperated and thus the old logic still largely applies. 
    Down the road we might want to redcue the number of input variables since this looks a bit confusing now 
    """

    def __init__(self, core, period_choice_dense_cores, indexer, base_draws_sol, optim_paras, options, dense):
        self.base_draws_sol = base_draws_sol
        self.core = core
        self.indexer = indexer
        self.is_inadmissible = super()._create_is_inadmissible(optim_paras, options)
        self.indices_of_child_states = super()._create_indices_of_child_states(
            optim_paras
        )
        self.period_choice_dense_cores = period_choice_dense_cores
        self.sub_state_spaces = {
            dense_dim: _SingleDimStateSpace(
                self.core,
                self.indexer,
                {key:self.period_choice_dense_cores[key] for key in self.period_choice_dense_cores.keys() if key[0]==dense_dim }
                self.base_draws_sol,
                optim_paras,
                options,
                dense_dim,
                dense_covariates,
                self.is_inadmissible,
                self.indices_of_child_states,
                self.slices_by_periods,
            )
            for dense_dim, dense_covariates in dense.items()
        }
        

    def get_attribute(self, attribute):
        return {
            key: sss.get_attribute(attribute)
            for key, sss in self.sub_state_spaces.items()
        }

    def get_attribute_from_choice_set_private(self, attribute, period):
                return {
            key: sss.set_attribute_choice_sets_private(attribute, period)
            for key, sss in self.sub_state_spaces.items()
        }
    
    def get_attribute_from_period_choice_set(self, attribute, period):
        return {
            key: sss.get_attribute_from_period(attribute, period)
            for key, sss in self.sub_state_spaces.items()
        }
    
    def get_continuation_values(self, period, indices=None):
        return {
            key: sss.get_continuation_values(period, indices)
            for key, sss in self.sub_state_spaces.items()
        }
    
    def set_attribute(self, attribute, value):
        for key, sss in self.sub_state_spaces.items():
            sss.set_attribute(attribute, value[key])

    def set_attribute_from_choice_set_private(self, attribute,choice_set, value):
         for key, sss in self.sub_state_spaces.items():
                sss.set_attribute_from_choice_set_private(attribute,choice_set,value[key])
                    
    
    def set_attribute_from_period_choice_set(self, attribute, period, choice_set, value):
        for key, sss in self.sub_state_spaces.items():
                sss.set_attribute_from_choice_set_private(attribute,choice_set,value[key])
        
        
    @property
    def states(self):
        return {key: sss.states for key, sss in self.sub_state_spaces.items()}

In [17]:
class _BaseStateSpace:
    """The base class of a state space.
    The base class includes some methods which should be available to both state spaces
    and are shared between multiple sub state spaces.
    """

    def _create_indices_of_child_states(self, optim_paras):
        """For each parent state get the indices of child states.
        During the backward induction, the ``expected_value_functions`` in the future
        period serve as the ``continuation_values`` of the current period. As the
        indices for child states never change, these indices can be precomputed and
        added to the state_space.
        Actually, the indices of the child states do not have to cover the last period,
        but it makes the code prettier and reduces the need to expand the indices in the
        estimation.
        """
        n_choices = len(optim_paras["choices"])
        n_choices_w_exp = len(optim_paras["choices_w_exp"])
        n_periods = optim_paras["n_periods"]
        n_states = self.core.shape[0]
        core_columns = create_core_state_space_columns(optim_paras)

        indices = np.full(
            (n_states, n_choices), INDEXER_INVALID_INDEX, dtype=INDEXER_DTYPE
        )

        # Skip the last period which does not have child states.
        for period in reversed(range(n_periods - 1)):
            states_in_period = self.core.query("period == @period")[
                core_columns
            ].to_numpy(dtype=np.int8)

            indices = _insert_indices_of_child_states(
                indices,
                states_in_period,
                self.indexer[period],
                self.indexer[period + 1],
                self.is_inadmissible,
                n_choices_w_exp,
                optim_paras["n_lagged_choices"],
            )

        return indices

## StateSpace Utils
This contains all functions that are required.


### The StateSpace

Next, the StateSpace. Define another class

In [19]:
class StateSpace:
    def __init__():
        pass

### Get indices of child states

Before, the state space indices above on the lhs should be converted to integers for simplicity.

Then, we need to modify the current indexer because for each state we have to store the index of the sub state space and the row of its successor. Solutions:

1. Same indexer, but indexing yields a tuple containing the index of the part and the row. But the indexer is consumes lots of unnecessary memory for sparse state spaces.
2. Something new: Take all states from one `SubStateSpace` and update the states choosing one choice from the choice set. Then, look for duplicates with states in the next period. It is currently not a feature of pd.duplicated to see which rows are connected as duplicates, but part of the implementation is [pd.factorize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html). Then, take only rows which have a duplicated factorial. Take the last (if you appended the future period) duplicated value. Save the choice set and row index from the future in to the former period. This even works if two states have the same successor.

The second solution is definitely the way to go in the future as the indexer of KW97 still takes at least 1GB or even more.

In [20]:
pd.factorize([(1, 2), (2, 3), (1, 2)])

(array([0, 1, 0], dtype=int64), array([(1, 2), (2, 3)], dtype=object))

### Calculation of covariates

### Solution

Then, we are ready to modify the solution.

1. Change `_create_choice_rewards` such that only the rewards of choices in the choice set are computed.
2. Implement a new `get_continuation_values` function for the `StateSpace`.
3. Run the backward induction.

I think we would want to loop over all period, state, dense combs and calculate their corresponding continuation values!
How should we realize that. Would we want these simple states to have 

In [ ]:
#Think

### Archive

In [35]:
# Maybe a hint for the performance if we need to collect continuation values from successors.
# The parts are not correct, but in total this should resemble KW97. We definitely take a hit here,
# but save it as memory and runtime in MCS in other parts. With jit, we might get some more performance.

n_parts = 4
n_states_per_part = 14_000_000

emax = {i: np.arange(n_states_per_part) for i in range(n_parts)}
continuation_values = np.empty((n_parts, n_states_per_part))

In [36]:
%%timeit

for i in range(n_parts):
    for j in range(n_states_per_part):
        continuation_values[i, j] = emax[i][j]

27.3 s ± 1.5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
# Same with Numba typed dicts (http://numba.pydata.org/numba-doc/dev/reference/pysupported.html#typed-dict).
# This cuts runtim to at least a fifth of the other approach.

from numba import njit, prange
from numba import types
from numba.typed import Dict

@njit(parallel=True)
def _get_continuation_values(emax, continuation_values):
    n_parts, n_states_per_part = continuation_values.shape
    
    for i in prange(n_parts):
        for j in prange(n_states_per_part):
            continuation_values[i, j] = emax[i][j]
            
    return continuation_values

In [69]:
%%timeit

# The Dict.empty() constructs a typed dictionary.
# The key and value typed must be explicitly declared.
emax = Dict.empty(
    key_type=types.int64,
    value_type=types.float64[:],
)

for i in range(n_parts):
    emax[i] = np.arange(n_states_per_part, dtype="float64")
continuation_values = np.empty((n_parts, n_states_per_part))

continuation_values = _get_continuation_values(emax, continuation_values)

4.27 s ± 132 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [70]:
_get_continuation_values.parallel_diagnostics(level=4)

 
 Parallel Accelerator Optimizing:  Function _get_continuation_values, <ipython-
input-68-ff857e4f6260> (8)  


Parallel loop listing for  Function _get_continuation_values, <ipython-input-68-ff857e4f6260> (8) 
--------------------------------------------------------------|loop #ID
@njit(parallel=True)                                          | 
def _get_continuation_values(emax, continuation_values):      | 
    n_parts, n_states_per_part = continuation_values.shape    | 
                                                              | 
    for i in prange(n_parts):---------------------------------| #6
        for j in prange(n_states_per_part):-------------------| #5
            continuation_values[i, j] = emax[i][j]            | 
                                                              | 
    return continuation_values                                | 
--------------------------------- Fusing loops ---------------------------------
Attempting fusion of parallel loops (combines 

In [67]:
# Convert tuples of booleans from choice set to actual choice names.

def choice_set_booleans_to_names(booleans, choices):
    return tuple(itertools.compress(choices, booleans))

choice_set_booleans_to_names([False, True], ["a", "b"])

('b',)